In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('/home/enis/projects/similarSoundsApp/'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/similarSoundsApp


In [8]:
from datetime import timedelta
import datetime 
import os


from nna.fileUtils import read_file_properties_v2,find_filesv2,list_files,get_labeled_exif
from nna.fileUtils import get_audio,str2timestamp,query_audio
from nna.labeling_utils import splitmp3

import pandas as pd
from pathlib import Path

from sklearn.neighbors import KDTree
import numpy as np

from pydub import AudioSegment


In [9]:
from annoy import AnnoyIndex


In [10]:
from sklearn.metrics.pairwise import paired_distances
from sklearn.metrics import pairwise_distances
from annoy import AnnoyIndex


def paired_distances_broadcast(X,Y):
    return paired_distances(np.repeat(X,Y.shape[0],axis=0), Y)


def sigmoid(X):
    return 1/(1+np.exp(-X))

def sigmoidT(X,t):
    return 1/(1+np.exp(-X*t))

def dist2sim(results,gamma=1/512):
    return np.exp(-results*gamma)

In [5]:
# output = find_filesv2(location,start_time,end_time,length,0,file_properties_df)
# sorted_filtered,start_time,end_time,start_time_org,end_time_org = output

#### find the embedding for a given image

In [6]:
# p_path="/home/enis/projects/nna/data/cameraTraps/test.txt"

# with open(p_path) as p_file:
#     photo_paths=p_file.readlines()
#     photo_paths = [i.strip() for i in photo_paths]

# other_folder=["anwr_35_2019_101RECNX_RCNX3373.JPG",
# "anwr_42_2019_100RECNX_RCNX3574.JPG",
# "ivvavik_SINP03_2019_100RECNX_IMG_3219.JPG",
# "ivvavik_SINP05_2019_100RECNX_IMG_2867.JPG",
# "ivvavik_SINP06_2019_100RECNX_IMG_1884.JPG",
# "ivvavik_SINP09_2019_100RECNX_IMG_2743.JPG",
# "prudhoe_17_2019_100RECNX_RCNX3916.JPG",]

# parent_path1="/tank/data/nna/examples/randomPhotos10k/"
# parent_path2="/tank/data/nna/examples/randomPhotos1k/"

# # photo with Caribou
# photo_paths=['anwr_37_2019_100RECNX_RCNX9317.jpg']

# given image paths finds files, code is at:
# notebooks/explore/get_audio4photos.ipynb

### 1. pick a sound to use for queries

In [11]:
# sound with Caribou

# !find /scratch/enis/data/nna/real/ -iname "S4A10279_20190605_091602*"

In [12]:
# aircraft sound

# !find /scratch/enis/data/nna/real/ -iname "S4A10255_20190507_073000*"


### 1.1 find similar embeddings for given embeddings in the same location 

In [31]:
# sound with Caribou

raw_embed_name="/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_vgg/S4A10279_20190605_091602_rawembeddings000.npy"
raw_embed=np.load(raw_embed_name)


In [9]:
# aircraft sound
# raw_embed_name="/scratch/enis/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000_vgg/S4A10255_20190507_073000_rawembeddings000.npy"
raw_embed_name="/scratch/enis/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000_vgg/S4A10255_20190507_073000_embeddings000.npy"

raw_embed=np.load(raw_embed_name)
raw_embed=raw_embed.astype(np.float32)


In [240]:
# S4A10255_20190507_073000-1808second
(30*60)+8


1808

### 1.2 Slice embedding for exact time of interest from a big file

In [10]:
# aircraft sound
start_seconds=(30*60)+8
raw_embed_audio=raw_embed[int(start_seconds):int(start_seconds)+60]

In [33]:

# # sound with Caribou
# start_seconds=(start_time-sorted_filtered["timestamp"])[0].total_seconds()
# raw_embed_audio=raw_embed[int(start_seconds):int(start_seconds)+60]

In [34]:
raw_embed_audio.shape

(60, 128)

In [16]:
from nnSearch import getFileIndex

In [17]:
file_properties_df=pd.read_pickle("../../data/allFields_dataV2.pkl")


In [18]:
import csv
metadatCsv="/scratch/enis/data/nna/realMerged/allEmbeddings_V11.csv"
file_properties_dfPath="/home/enis/projects/nna/data/allFields_dataV3.pkl"

metadata=[]
with open(metadatCsv) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        metadata.append((row[0],int(row[1])))

# NNIndexbyAnnoy = loadOnDisk(fn,f,distance=distance)
file_properties_df=pd.read_pickle(file_properties_dfPath)


In [19]:

from nna.fileUtils import standard_path_style

thePath=Path(metadata[445][0])
parentDistance=0
thePath.parents[parentDistance],thePath
outputPath="/scratch/enis/data/nna/real/"
inputPath="/tank/data/nna/real/"

from nna.fileUtils import npy2originalfile

npy2originalfilethePath,outputPath,file_properties_df)

PosixPath('/tank/data/nna/real/anwr/31/2019/S4A10297_20190616_124602.flac')

In [20]:
metadata[445][0],445

('/scratch/enis/data/nna/real/anwr/31/2019/S4A10297_20190616_124602_vgg/S4A10297_20190616_124602_embeddings000.npy',
 445)

In [21]:
index=5
i=44589
ii=i
meta=metadata[i]

distance=int(Path(meta[0]).stem[-3:])
secondsInOriginalFile=0
for m in range(i-1,i-distance-1,-1):
    secondsInOriginalFile+=metadata[m][1]
secondsInOriginalFile
# while True:
#     if Path(meta[0]).stem[-3:]=="000":
#         print(meta[0])
#         break
#     else:
#         ii-=1
#         meta=metadata[ii]

0

In [23]:
getFileIndex(raw_embed_name,metadata,secondsfromBeginningofFile=0)

In [24]:
for i,m in enumerate(metadata):
    if "001.npy" in m[0]:
        print(m,i)
        break

('/scratch/enis/data/nna/real/stinchcomb/01-Itkillik/2016/ITK01_20160602_130218_vgg/ITK01_20160602_130218_embeddings001.npy', 3600) 44590


### 2.1 load embeddings to search from 

In [200]:
%%time
# find all files
import glob
search_path="/scratch/enis/data/nna/real/prudhoe/26/"
metadataCSV="/scratch/enis/data/nna/realMerged/"+"allEmbeddings_"+"prudhoe26_V11"+".csv"

filenamePattern="*_embeddings*.npy"

# all_embeddings=list_files("/scratch/enis/data/nna/real/anwr/37/",filenamePattern)
# aircraft sound
all_embeddings=list_files(search_path,filename=filenamePattern)
# remove original embedding from the list
# del all_embeddings[all_embeddings.index(raw_embed_name)]

embedSizes=[]
for i,embed_file in enumerate(all_embeddings):
    embed=np.load(embed_file)
    embedSizes.append(embed.shape[0])

metadata=list(zip(all_embeddings,embedSizes))
save_to_csv(metadataCSV,metadata)

386
CPU times: user 748 ms, sys: 1.82 s, total: 2.57 s
Wall time: 5min 56s


* we are trying to create a matrix, rows are queries and columns are data points in database
* values are similarity, 
* then 


### 2.1.1 this is for calculating distances ourself 

In [45]:
%%time
# load and merge embedding files
#LOAD:
embeds=[None]*len(all_embeddings)
index=0
for i,embed_file in enumerate(all_embeddings):
    embed=np.load(embed_file)
    index+=embed.shape[0]
    embeds[i]=(embed)
#MERGE:
# Faster then np.concatenate
concat_embeds=np.zeros([index,128],dtype=np.float32)
index=0
for i,embed in enumerate(embeds):
    concat_embeds[index:(index+embed.shape[0]),:]=embed[:]
    index+=embed.shape[0]

CPU times: user 996 ms, sys: 1 s, total: 2 s
Wall time: 4 s


In [117]:
concat_embeds.shape

(1736614, 128)

### 2.1.2 create an ANNOY index for files at all_path on disk

In [11]:
# vector size
f=128
  # Length of item vector that will be indexed
distance='euclidean'
#prepares annoy to build the index in the specified file instead of RAM 
#(execute before adding items, no need to save after build)
fn="/scratch/enis/data/nna/realMerged/prudhoe26_V11.ann"
fn="/scratch/enis/data/nna/realMerged/allEmbeddingsV11.ann"


In [4]:

import time


from nnSearch import loadOnDisk,buildOnDisk


In [5]:

p=Path(fn)
if p.exists():
    t = loadOnDisk(fn,f,distance=distance)
else:
    t = buildOnDisk(fn,f,distance=distance,seed=42,treeN=16)

NameError: name 'Path' is not defined

In [ ]:
import numpy as np
import csv
from annoy import AnnoyIndex

from nnSearch import MatrixApproxV2,groupResByDistance,embedIndex2fileSecond



# vector size
f=128
  # Length of item vector that will be indexed
distance='euclidean'
#prepares annoy to build the index in the specified file instead of RAM 
#(execute before adding items, no need to save after build)
fn="/scratch/enis/data/nna/realMerged/allEmbeddingsV11.ann"
metadatCsv="/scratch/enis/data/nna/realMerged/allEmbeddingsV11.csv"
t = loadOnDisk(fn,f,distance=distance)

metadata=[]
with open(metadatCsv) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        metadata.append((row[0],int(row[1])))


aircraftFileIndex=getFileIndex(raw_embed_name,metadata)
startIndex=aircraftFileIndex+1808

queries=[i for i in range(startIndex,startIndex+10,1)]
searchDepth=2

QmatrixDistance,QmatrixIDs=MatrixApproxV2(queries,t,searchDepth=2)
averageDistance_approx=np.average(QmatrixDistance,axis=1)

similarities,IDs=dist2sim(averageDistance_approx).flatten(),QmatrixIDs[:,:,0].flatten()
res=list(zip(similarities,IDs))

ResGrouped=groupResByDistance(res,theta=5)
split_folder="/home/enis/projects/nna/data/nearestNeighbours/prudhoe26_ApproxV2/"

exp_name=""
for indexes in (ResGrouped):
    score,index=(indexes[0])
    groupLength=indexes[-1][1]-indexes[0][1]+10
#     print(groupLength)
#     print(score,index)

    clipAddBip([index],exp_name,split_folder,metadata,excerptLen=groupLength,reductionLen=1)



In [329]:
# sort results by similarity 
scores=[max([i[0] for i in x]) for x in ResGrouped]
ResGroupedwScores=list(zip(scores,ResGrouped))
ResGroupedwScores.sort(key=lambda x: x[0],reverse=True)
ResGrouped=[i[1] for i in ResGroupedwScores]


### 3.1 Use Annoy NNs to approximate matrix, (find highest per second, then calculate others)

In [255]:
similarities,IDs=dist2sim(averageDistance_approx).flatten(),QmatrixIDs[:,:,0].flatten()
res=list(zip(similarities,IDs))


In [256]:
len(res)

20

#### Highest scores might be consequtive, merging these as one result helps decrease amount of files to listen

In [6]:
from nnSearch import groupResByDistance




In [257]:
ResGrouped=groupResByDistance(res,theta=5)
# print("total groups",len(ResGrouped))
# previous=0
# print("short distances")
# for score_indexes in ResGrouped:
#     if (score_indexes[-1][1]-score_indexes[0][1])>30:
#         print(score_indexes[0][1],score_indexes[-1][1])


In [258]:
ResGrouped

[[(0.3420873627434949, 199662)],
 [(0.23513666278291756, 199763),
  (0.24773782430353594, 199765),
  (0.25793467015829874, 199765)],
 [(0.22867374500591423, 199775)],
 [(0.3133193261908106, 199898), (0.26999331052534553, 199899)],
 [(0.3049009514846011, 208611)],
 [(0.21640606362211398, 274935),
  (0.24928697352579735, 274935),
  (0.22077357382274507, 274936)],
 [(0.3502257485041737, 398432)],
 [(0.2747582949889767, 403581)],
 [(0.2555197891287858, 828917)],
 [(0.3149978497165593, 879141)],
 [(0.1842298181046157, 1051635), (0.24000938942737535, 1051636)],
 [(0.2830081520104452, 1674383)],
 [(0.22750286629790317, 1686194)],
 [(0.36534653708099707, 1686206)]]

In [259]:
ResGrouped=groupResByDistance(res,theta=5)
split_folder="/home/enis/projects/nna/data/nearestNeighbours/prudhoe26_ApproxV2/"

exp_name=""
for indexes in (ResGrouped):
    score,index=(indexes[0])
    groupLength=indexes[-1][1]-indexes[0][1]+10
    print(groupLength)
    print(score,index)

    clipAddBip([index],exp_name,split_folder,metadata,excerptLen=groupLength,reductionLen=1)

# indexes=[indexes[len(indexes)//2] for indexes,count in (starts)]
# # indexes
    


10
0.3420873627434949 199662
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000.flac 1706
12
0.23513666278291756 199763
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000.flac 1807
10
0.22867374500591423 199775
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000.flac 1819
11
0.3133193261908106 199898
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000.flac 1942
10
0.3049009514846011 208611
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190507_120000.flac 1657
11
0.21640606362211398 274935
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190508_221602.flac 435
10
0.3502257485041737 398432
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190511_150000.flac 2520
10
0.2747582949889767 403581
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190511_161602.flac 3109
10
0.2555197891287858 828917
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190521_003000.flac 1101
10
0.3149978497165593 879141
/tank/data/nna/real/prudhoe/26/2019/S4A10255_20190522_011602.flac 177

4562

### 3.1.1 lets calculate exact values with ANNOY to see the difference

In [ ]:
%%time
matrixList=[]
queries=[i for i in range(100,110,1)]
for i,q in enumerate(queries):
    IDs,Distances=t.get_nns_by_item(q, sizeOfDataset,include_distances=True)
    a=list(zip(IDs,Distances))
    a.sort()
    print(i)
    matrixList.append(a[:])

for a in matrixList:
    print(len(a))

matrixList2=[]
for a in matrixList:
    b=[i[1] for i in a]
    matrixList2.append(b)
bb=np.array(matrixList2)

gamma=1/450
results=np.exp(-bb*gamma)

%%time
windowSize=10
# here I used average, normally we were using np.max
windowMax=np.average(results,axis=0)
windowMean=pd.Series(windowMax).rolling(window=windowSize).mean().iloc[windowSize-1:].values

# arr = np.array([1, 3, 2, 4, 5,6,7,8,8,1,1,9])
kth=40
ind = np.argpartition(windowMean, -kth)[-kth:]
sortedbyScore=sorted(list(zip(windowMean[ind],ind)),reverse=True)

for _,index in sortedbyScore[10:]:
    
    print(np.argmax(bb[:,index:index+10],axis=0))


### 4.1.0 there are three methods for creating vectors (I was experimenting merging vectors)

In [15]:
# # Method 1   1 second 
excerptLen=1
exp_name="1second"
concat_embeds2=concat_embeds[:]
raw_embed_audio2=raw_embed_audio[:]
raw_embed_audio2=raw_embed_audio2[20:21,:]
rowN=100000

#Method 2    mean 10 second
# excerptLen=10
# exp_name="Mean10Second"
# concat_embeds2=concat_embeds.reshape(-1,10,128).mean(axis=1)
# raw_embed_audio2=raw_embed_audio.reshape(-1,10,128).mean(axis=1)
# raw_embed_audio2=raw_embed_audio2[2:3,:]
# rowN=10000

# #Method 3    concat 10 second
# excerptLen=10
# exp_name="Concat10Second"
# concat_embeds2=concat_embeds.reshape(-1,1280)
# raw_embed_audio2=raw_embed_audio.reshape(-1,1280)
# raw_embed_audio2=raw_embed_audio2[2:3,:]
# rowN=10000



#### 4.1.1 Exp: Fastest Way to calculate exact matrix, 

In [69]:
%%time
import multiprocessing

def worker(index,numberItems):
    allA=[None]*numberItems

    for i in range(numberItems):
        a=u.get_distance(i, index)
    return None

jobs = []
for index in range(10):
    numberItems=u.get_n_items()
    p = multiprocessing.Process(target=worker,args=(index,numberItems))
    jobs.append(p)
    p.start()

for p in jobs:
    p.join()

CPU times: user 52 ms, sys: 788 ms, total: 840 ms
Wall time: 2.08 s


In [119]:
%%time
# one sample/query
# res=paired_distances_broadcast(raw_embed_audio2,concat_embeds2)
# res.shape

#
# 10 queries
queryCount=10
startSecond=0
excerptLen=1

gamma=1/512
startIndex=int(startSecond/excerptLen)
endIndex=startIndex+queryCount


results = pairwise_distances(raw_embed_audio[startIndex:10,:], concat_embeds)

# results=np.exp(-results*gamma)



CPU times: user 2min 55s, sys: 17.2 s, total: 3min 12s
Wall time: 5.09 s


In [169]:
%%time
results = dist2sim(results,gamma)


CPU times: user 472 ms, sys: 2.57 s, total: 3.04 s
Wall time: 3.05 s


In [32]:
# I might have to use this method, if I want to get maximum without replacement

# %%time
# stepSize=1
# windowSize=10
# resultsMax=[]
# resultsAvg=[]

# windowMax=np.max(results2,axis=0)

# for i in range(0,results2.shape[1],stepSize):
# #     windowElements=(results2[:,i:(i+windowSize+1)])
# #     colMax=np.max(windowElements,axis=0)

#     resultAvg=np.average(colMax)
#     resultsAvg.append(resultAvg)
# #     resultsMax.append(windowMax)


CPU times: user 29.9 s, sys: 960 ms, total: 30.8 s
Wall time: 29.8 s


### 4.1.2 Run sliding window over the exact Matrix

In [491]:
%%time
windowSize=10
windowMax=np.max(results,axis=0)
windowMean=pd.Series(windowMax).rolling(window=windowSize).mean().iloc[windowSize-1:].values


CPU times: user 2.31 s, sys: 1min 3s, total: 1min 5s
Wall time: 1min 5s


In [ ]:
%%time
windowSize=10
windowMax=np.max(resultMatrix,axis=0)
windowMean=pd.Series(windowMax).rolling(window=windowSize).mean().iloc[windowSize-1:].values


#### Look into stats of results

In [203]:
from scipy import stats

stats.describe(res)

DescribeResult(nobs=1732041, minmax=(0.05875636413693428, 0.3321208477020264), mean=0.12500929506519953, variance=0.00040337682893698085, skewness=1.7658461475250447, kurtosis=8.105885715994328)

In [201]:
res=windowMean[:]

#### find kth highest scores (similarity of queries)

In [334]:
# arr = np.array([1, 3, 2, 4, 5,6,7,8,8,1,1,9])
kth=250
ind = np.argpartition(res, -kth)[-kth:]
sortedbyScore=sorted(list(zip(res[ind],ind)),reverse=True)

### generating figures for similarity scores

In [152]:
res.shape,rowN,res.shape[0]/rowN

((1732041,), 100000, 17.32041)

In [153]:
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

minY,maxY=min(res),max(res)
exp_name="slidingWindow_AverageOfMax_short20k"
name=f"Euclidian Similarity {exp_name}"
nrows=res.shape[0]//rowN
print(nrows)
fig, ax = plt.subplots(nrows=nrows,figsize=(200, 20*nrows))
for i,axes in enumerate(ax):
    axes.plot(res[i*(rowN):(i+1)*(rowN)])
    axes.set_ylim(minY,maxY)
    axes.grid(True)

ax[0].set_ylabel(f"{name}",fontsize=32)

# plt.grid(True)

# loc = plticker.MultipleLocator(base=5000) # this locator puts ticks at regular intervals
# ax.xaxis.set_major_locator(loc)

plt.tight_layout()
fig.show()
filename=name.replace(" ","_")
fig.savefig(f"/home/enis/projects/nna/results/vis/nearestNeighbour/{filename}.png")


17


In [169]:
excerptLen=10
exp_name="Concat10Second"
concat_embeds2=concat_embeds.reshape(-1,1280)
raw_embed_audio2=raw_embed_audio.reshape(-1,1280)
raw_embed_audio2=raw_embed_audio2[2:3,:]
rowN=10000

res=paired_distances_broadcast(raw_embed_audio2,concat_embeds2)
res.shape

import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

name=f"Euclidian Distance {exp_name}"
nrows=res.shape[0]//rowN

fig, ax = plt.subplots(nrows=nrows,figsize=(200, 20*nrows))
for i,axes in enumerate(ax):
    axes.plot(res[i*(rowN):(i+1)*(rowN)])

ax[0].set_ylabel(f"{name}",fontsize=32)

plt.grid(True)

# loc = plticker.MultipleLocator(base=5000) # this locator puts ticks at regular intervals
# ax.xaxis.set_major_locator(loc)

plt.tight_layout()
# fig.show()
filename=name.replace(" ","_")
fig.savefig(f"/home/enis/projects/nna/results/vis/nearestNeighbour/{filename}.png")


In [232]:
len(all_embeddings),excerptLen

(385, 10)

In [310]:
# sorted([sorted(start) for start in starts])

### create corresponding clips

In [ ]:
split_folder="/home/enis/projects/nna/data/nearestNeighbours/S4A10255_20190507_073000-1808seconds/slidingWindow_AverageOfMax/"
excerptLen=1
# for indexes in (starts):
#     score,index=(indexes[len(indexes)//2])
#     print(score)
#     print(embedIndex2fileSecond(index,embeds,all_embeddings,excerptLen=1))
    
for score,index in (sortedbyScore):
#     score,index=(indexes[len(indexes)//2])
    print(score)
    print(embedIndex2fileSecond(index,embeds,all_embeddings,excerptLen=1))

# indexes=[indexes[len(indexes)//2] for indexes,count in (starts)]
# # # indexes
# clipAddBip(indexes,exp_name,split_folder,embeds,all_embeddings,excerptLen=10,reductionLen=1)
    


In [50]:
kdt = KDTree(concat_embeds2, leaf_size=30, metric='euclidean')
distances,indexes=kdt.query(raw_embed_audio2, k=10, return_distance=True)
allResults=[ (distance,index) for index,distance in (zip(indexes.flatten(),distances.flatten()))]
allResults.sort()
# distances,indexes=allResults

In [209]:
from nnSearch import embedIndex2fileSecond

In [210]:
split_folder="/home/enis/projects/nna/data/nearestNeighbours/"
from nnSearch import clipAddBip




In [7]:
#notes
!ls -alh /scratch/enis/data/nna/realMerged/allEmbeddings.ann

ls: cannot access '/scratch/enis/data/nna/realMerged/allEmbeddings.ann': No such file or directory
